# Movie Recommendations HW

**Name:**  

**Collaboration Policy:** Homeworks will be done individually: each student must hand in their own answers. Use of partial or entire solutions obtained from others or online is strictly prohibited.

**Late Policy:** Late submission have a penalty of 2\% for each passing hour. 

**Submission format:** Successfully complete the Movie Lens recommender as described in this jupyter notebook. Submit a `.py` and an `.ipynb` file for this notebook. You can go to `File -> Download as ->` to download a .py version of the notebook. 

**Only submit one `.ipynb` file and one `.py` file.** The `.ipynb` file should have answers to all the questions. Do *not* zip any files for submission. 

**Download the dataset from here:** https://grouplens.org/datasets/movielens/1m/

In [1]:
# Import all the required libraries
import numpy as np
import pandas as pd
from  numpy.linalg import matrix_rank as rank
import math
def compute_slice(U, S, VT, r):
    Xapprox = U[:,:r]@S[:r,:r]@VT[:r,:]
    return Xapprox

## Reading the Data
Now that we have downloaded the files from the link above and placed them in the same directory as this Jupyter Notebook, we can load each of the tables of data as a CSV into Pandas. Execute the following, provided code.

In [2]:
# Read the dataset from the two files into ratings_data and movies_data
#NOTE: if you are getting a decode error, add "encoding='ISO-8859-1'" as an additional argument
#      to the read_csv function
column_list_ratings = ["UserID", "MovieID", "Ratings","Timestamp"]
ratings_data  = pd.read_csv('ratings.dat',sep='::',names = column_list_ratings, engine='python')
column_list_movies = ["MovieID","Title","Genres"]
movies_data = pd.read_csv('movies.dat',sep = '::',names = column_list_movies, engine='python', encoding = 'latin-1')
column_list_users = ["UserID","Gender","Age","Occupation","Zixp-code"]
user_data = pd.read_csv("users.dat",sep = "::",names = column_list_users, engine='python')

`ratings_data`, `movies_data`, `user_data` corresponds to the data loaded from `ratings.dat`, `movies.dat`, and `users.dat` in Pandas.

## Data analysis

We now have all our data in Pandas - however, it's as three separate datasets! To make some more sense out of the data we have, we can use the Pandas `merge` function to combine our component data-frames. Run the following code:

In [3]:
data=pd.merge(pd.merge(ratings_data,user_data),movies_data)
data

,UserID,MovieID,Ratings,Timestamp,Gender,Age,Occupation,Zixp-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,M,18,17,47901,Modulations (1998),Documentary
1000205,5675,2703,3,976029116,M,35,14,30030,Broken Vessels (1998),Drama
1000206,5780,2845,1,958153068,M,18,17,92886,White Boys (1999),Drama
1000207,5851,3607,5,957756608,F,18,20,55410,One Little Indian (1973),Comedy|Drama|Western


Next, we can create a pivot table to match the ratings with a given movie title. Using `data.pivot_table`, we can aggregate (using the average/`mean` function) the reviews and find the average rating for each movie. We can save this pivot table into the `mean_ratings` variable. 

In [4]:
mean_ratings=data.pivot_table('Ratings','Title',aggfunc='mean')
mean_ratings

,Ratings
Title,
"$1,000,000 Duck (1971)",3.027027
'Night Mother (1986),3.371429
'Til There Was You (1997),2.692308
"'burbs, The (1989)",2.910891
...And Justice for All (1979),3.713568
...,...
"Zed & Two Noughts, A (1985)",3.413793
Zero Effect (1998),3.750831
Zero Kelvin (Kjærlighetens kjøtere) (1995),3.500000


Now, we can take the `mean_ratings` and sort it by the value of the rating itself. Using this and the `head` function, we can display the top 15 movies by average rating.

In [5]:
mean_ratings=data.pivot_table('Ratings',index=["Title"],aggfunc='mean')
top_15_mean_ratings = mean_ratings.sort_values(by = 'Ratings',ascending = False).head(15)
top_15_mean_ratings

,Ratings
Title,
Ulysses (Ulisse) (1954),5.000000
Lured (1947),5.000000
Follow the Bitch (1998),5.000000
Bittersweet Motel (2000),5.000000
Song of Freedom (1936),5.000000
One Little Indian (1973),5.000000
Smashing Time (1967),5.000000
Schlafes Bruder (Brother of Sleep) (1995),5.000000
"Gate of Heavenly Peace, The (1995)",5.000000


Let's adjust our original `mean_ratings` function to account for the differences in gender between reviews. This will be similar to the same code as before, except now we will provide an additional `columns` parameter which will separate the average ratings for men and women, respectively.

In [6]:
mean_ratings=data.pivot_table('Ratings',index=["Title"],columns=["Gender"],aggfunc='mean')
mean_ratings

Gender,F,M
Title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
...,...,...
"Zed & Two Noughts, A (1985)",3.500000,3.380952
Zero Effect (1998),3.864407,3.723140
Zero Kelvin (Kjærlighetens kjøtere) (1995),NaN,3.500000


We can now sort the ratings as before, but instead of by `Rating`, but by the `F` and `M` gendered rating columns. Print the top rated movies by male and female reviews, respectively.

In [7]:
data=pd.merge(pd.merge(ratings_data,user_data),movies_data)

mean_ratings=data.pivot_table('Ratings',index=["Title"],columns=["Gender"],aggfunc='mean')
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
print(top_female_ratings.head(15))

top_male_ratings = mean_ratings.sort_values(by='M', ascending=False)
print(top_male_ratings.head(15))

Gender                                               F         M
Title                                                           
Clean Slate (Coup de Torchon) (1981)               5.0  3.857143
Ballad of Narayama, The (Narayama Bushiko) (1958)  5.0  3.428571
Raw Deal (1948)                                    5.0  3.307692
Bittersweet Motel (2000)                           5.0       NaN
Skipped Parts (2000)                               5.0  4.000000
Lamerica (1994)                                    5.0  4.666667
Gambler, The (A Játékos) (1997)                    5.0  3.166667
Brother, Can You Spare a Dime? (1975)              5.0  3.642857
Ayn Rand: A Sense of Life (1997)                   5.0  4.000000
24 7: Twenty Four Seven (1997)                     5.0  3.750000
Twice Upon a Yesterday (1998)                      5.0  3.222222
Woman of Paris, A (1923)                           5.0  2.428571
I Am Cuba (Soy Cuba/Ya Kuba) (1964)                5.0  4.750000
Gate of Heavenly Peace, T

In [8]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']
sorted_by_diff = mean_ratings.sort_values(by='diff')
sorted_by_diff[:10]

Gender,F,M,diff
Title,,,
"James Dean Story, The (1957)",4.000000,1.000000,-3.000000
Country Life (1994),5.000000,2.000000,-3.000000
"Spiders, The (Die Spinnen, 1. Teil: Der Goldene See) (1919)",4.000000,1.000000,-3.000000
Babyfever (1994),3.666667,1.000000,-2.666667
"Woman of Paris, A (1923)",5.000000,2.428571,-2.571429
Cobra (1925),4.000000,1.500000,-2.500000
"Other Side of Sunday, The (Søndagsengler) (1996)",5.000000,2.928571,-2.071429
"To Have, or Not (1995)",4.000000,2.000000,-2.000000
For the Moment (1994),5.000000,3.000000,-2.000000


Let's try grouping the data-frame, instead, to see how different titles compare in terms of the number of ratings. Group by `Title` and then take the top 10 items by number of reviews. We can see here the most popularly-reviewed titles.

In [9]:
ratings_by_title=data.groupby('Title').size()
ratings_by_title.sort_values(ascending=False).head(10)

Title
American Beauty (1999)                                   3428
Star Wars: Episode IV - A New Hope (1977)                2991
Star Wars: Episode V - The Empire Strikes Back (1980)    2990
Star Wars: Episode VI - Return of the Jedi (1983)        2883
Jurassic Park (1993)                                     2672
Saving Private Ryan (1998)                               2653
Terminator 2: Judgment Day (1991)                        2649
Matrix, The (1999)                                       2590
Back to the Future (1985)                                2583
Silence of the Lambs, The (1991)                         2578
dtype: int64

Similarly, we can filter our grouped data-frame to get all titles with a certain number of reviews. Filter the dataset to get all movie titles such that the number of reviews is >= 2500.

## Question 1

Create a ratings matrix using Numpy. This matrix allows us to see the ratings for a given movie and user ID. The element at location $[i,j]$ is a rating given by user $i$ for movie $j$. Print the **shape** of the matrix produced.  

Additionally, choose 3 users that have rated the movie with MovieID "**1377**" (Batman Returns). Print these ratings, they will be used later for comparison.


**Notes:**
- Do *not* use `pivot_table`.
- A ratings matrix is *not* the same as `ratings_data` from above.
- The ratings of movie with MovieID $i$ are stored in the ($i$-1)th column (index starts from 0)  
- Not every user has rated every movie. Missing entries should be set to 0 for now.
- If you're stuck, you might want to look into `np.zeros` and how to use it to create a matrix of the desired shape.
- Every review lies between 1 and 5, and thus fits within a `uint8` datatype, which you can specify to numpy.

In [10]:
# Create the matrix
ratings = np.zeros((len(user_data), movies_data.max()[0]))
batmanusers = [[],[]]
for i in range(len(ratings_data)):
    userid = (int)(ratings_data.at[ratings_data.index[i], 'UserID'])
    movieid = (int)(ratings_data.at[ratings_data.index[i], 'MovieID'])
    ratingnum = (ratings_data.at[ratings_data.index[i], 'Ratings']).astype('uint8')
    ratings[userid-1][movieid-1] = ratingnum
    if movieid == 1377 and len(batmanusers[0]) < 3:
        batmanusers[0].append(userid)
        batmanusers[1].append(ratingnum)

In [11]:
# Print the shape
print("Shape of Ratings: ", ratings.shape)

Shape of Ratings:  (6040, 3952)


In [12]:
# Store and print ratings for Batman Returns
print("Batman Returns Users and Reviews: ", batmanusers)

Batman Returns Users and Reviews:  [[10, 13, 18], [3, 3, 2]]


## Question 2

Normalize the ratings matrix (created in **Question 1**) using Z-score normalization. While we can't use `sklearn`'s `StandardScaler` for this step, we can do the statistical calculations ourselves to normalize the data.

Before you start:
- Your first step should be to get the average of every *column* of the ratings matrix (we want an average by title, not by user!).
- Make sure that the mean is calculated considering only non-zero elements. If there is a movie which is rated only by 10 users, we get its mean rating using (sum of the 10 ratings)/10 and **NOT** (sum of 10 ratings)/(total number of users)
- All of the missing values in the dataset should be replaced with the average rating for the given movie. This is a complex topic, but for our case replacing empty values with the mean will make it so that the absence of a rating doesn't affect the overall average, and it provides an "expected value" which is useful for computing correlations and recommendations in later steps.
- In our matrix, 0 represents a missing rating.
- Next, we want to subtract the average from the original ratings thus allowing us to get a mean of 0 in every *column*. It may be very close but not exactly zero because of the limited precision `float`s allow.
- Lastly, divide this by the standard deviation of the *column*.

- Not every MovieID is used, leading to zero columns. This will cause a divide by zero error when normalizing the matrix. Simply replace any NaN values in your normalized matrix with 0.

In [14]:
ratings[ratings == 0] = np.nan
means = np.nanmean(ratings, axis=0)
meanslist = np.ndarray.tolist(means)
ratings = ratings - meanslist
ratings[np.isnan(ratings)] = 0
std = np.std(ratings, axis=0)
stdlist = np.ndarray.tolist(std)
ratings = ratings/stdlist
ratings[np.isnan(ratings)] = 0

C:\msys64\tmp\ipykernel_26076\4092891929.py:2: RuntimeWarning: Mean of empty slice
  means = np.nanmean(ratings, axis=0)
C:\msys64\tmp\ipykernel_26076\4092891929.py:8: RuntimeWarning: invalid value encountered in divide
  ratings = ratings/stdlist


## Question 3

We're now going to perform Singular Value Decomposition (SVD) on the normalized ratings matrix from the previous question. Perform the process using numpy, and along the way print the shapes of the $U$, $S$, and $V$ matrices you calculated.

In [15]:
# Compute the SVD of the normalised matrix
U, S, VT = np.linalg.svd(ratings)
S = np.diag(S)

In [16]:
# Print the shapes
print("U Shape: ", U.shape)
print("S Shape: ", S.shape)
print("VT Shape: ", VT.shape)

U Shape:  (6040, 6040)
S Shape:  (3952, 3952)
VT Shape:  (3952, 3952)


## Question 4

Reconstruct four rank-k rating matrix $R_k$, where $R_k = U_kS_kV_k^T$ for k = [100, 1000, 2000, 3000]. Using each of $R_k$ make predictions for the 3 users selected in Question 1, for the movie with ID 1377 (Batman Returns). Compare the original ratings with the predicted ratings.

In [17]:
ranks = [100, 1000, 2000, 3000]
rank100 = compute_slice(U=U, S=S, VT=VT, r=ranks[0])
rank1000 = compute_slice(U=U, S=S, VT=VT, r=ranks[1])
rank2000 = compute_slice(U=U, S=S, VT=VT, r=ranks[2])
rank3000 = compute_slice(U=U, S=S, VT=VT, r=ranks[3])
print("Original Ratings: ", batmanusers[1])
rank100ratings = []
rank1000ratings = []
rank2000ratings = []
rank3000ratings = []
for i in range(3):
    rank100ratings.append(rank100[batmanusers[0][i]-1][1376] * stdlist[1376] + meanslist[1376])
    rank1000ratings.append(rank1000[batmanusers[0][i]-1][1376] * stdlist[1376] + meanslist[1376])
    rank2000ratings.append(rank2000[batmanusers[0][i]-1][1376] * stdlist[1376] + meanslist[1376])
    rank3000ratings.append(rank3000[batmanusers[0][i]-1][1376] * stdlist[1376] + meanslist[1376])
#print(ratings[9][1376] * stdlist[1376] + meanslist[1376])
print("Rank100 Ratings: ", rank100ratings)
print("Rank1000 Ratings: ", rank1000ratings)
print("Rank2000 Ratings: ", rank2000ratings)
print("Rank3000 Ratings: ", rank3000ratings)

Original Ratings:  [3, 3, 2]
Rank100 Ratings:  [1.2927237774894307, -0.13695479878666061, 0.11569952478496938]
Rank1000 Ratings:  [0.04396383685249311, 0.17729125388938702, -1.2342892368113192]
Rank2000 Ratings:  [0.3763548778954648, 0.3157902824128648, -2.503631304168069]
Rank3000 Ratings:  [0.08743557396577108, 0.1297869314692908, -2.478213447516384]


## Question 5

### Cosine Similarity
Cosine similarity is a metric used to measure how similar two vectors are. Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space. Cosine similarity is high if the angle between two vectors is 0, and the output value ranges within $cosine(x,y) \in [0,1]$. $0$ means there is no similarity (perpendicular), where $1$ (parallel) means that both the items are 100% similar.

$$ cosine(x,y) = \frac{x^T y}{||x|| ||y||}  $$

**Based on the reconstruction rank-1000 rating matrix $R_{1000}$ and the cosine similarity,** sort the movies which are most similar. You will have a function `top_movie_similarity` which sorts data by its similarity to a movie with ID `movie_id` and returns the top $n$ items, and a second function `print_similar_movies` which prints the titles of said similar movies. Return the top 5 movies for the movie with ID `1377` (*Batman Returns*)

Note: While finding the cosine similarity, there are a few empty columns which will have a magnitude of **zero** resulting in NaN values. These should be replaced by 0, otherwise these columns will show most similarity with the given movie. 

In [18]:
# Sort the movies based on cosine similarity
def top_movie_similarity(data, movie_id, top_n=5):
    similarity = []
    for i in range(len(data[0])):
        ph = data[:, movie_id - 1]
        xt = np.transpose(ph)
        curMovie = data[:, i]
        sim = (np.dot(xt, curMovie))/(np.linalg.norm(ph)*np.linalg.norm(curMovie))
        if(np.isnan(sim)):
            sim = 0
        similarity.append(sim)
    negsim = [-x for x in similarity]
    similarity = np.argsort(negsim)
    return similarity
def print_similar_movies(movie_titles, top_indices):
    movienames = []
    for i in range(6):
        rownum = int(movie_titles[movie_titles['MovieID']==top_indices[i] + 1].index[0])
        movietitle = str(movie_titles.at[rownum, 'Title'])
        movienames.append((movietitle))
    print('Most Similar movies: ', movienames)

# Print the top 5 movies for Batman Returns
movie_id = 1377
print_similar_movies(movies_data, top_movie_similarity(rank1000, movie_id))

Most Similar movies:  ['Batman Returns (1992)', 'Batman Forever (1995)', 'Batman (1989)', 'Dick Tracy (1990)', 'Batman & Robin (1997)', 'Back to the Future Part II (1989)']


C:\msys64\tmp\ipykernel_26076\3481288428.py:8: RuntimeWarning: invalid value encountered in double_scalars
  sim = (np.dot(xt, curMovie))/(np.linalg.norm(ph)*np.linalg.norm(curMovie))


## Question 6

### Movie Recommendations
Using the same process from Question 5, write `top_user_similarity` which sorts data by its similarity to a user with ID `user_id` and returns the top result. Then find the MovieIDs of the movies that this similar user has rated most highly, but that `user_id` has not yet seen. Find at least 5 movie recommendations for the user with ID `5954` and print their titles.

Hint: To check your results, find the genres of the movies that the user likes and compare with the genres of the recommended movies.

In [19]:
#Sort users based on cosine similarity
def top_user_similarity(data, user_id):
    similarity = []
    for i in range(len(data)):
        ph = data[user_id - 1]
        xt = np.transpose(ph)
        curUser = data[i]
        sim = (np.dot(xt, curUser))/(np.linalg.norm(ph)*np.linalg.norm(curUser))
        if(np.isnan(sim)):
            sim = 0
        similarity.append(sim)
    negsim = [-x for x in similarity]
    similarity = np.argsort(negsim)
    user = data[similarity[1]]
    neguser = [-x for x in user]
    topusermovies = np.argsort(neguser)
    return topusermovies
print_similar_movies(movies_data, top_user_similarity(rank1000, 5954))

Most Similar movies:  ['Tarzan and the Lost City (1998)', 'Simon Sez (1999)', 'Loaded Weapon 1 (1993)', 'Avengers, The (1998)', 'Godzilla (1998)', 'Rising Sun (1993)']
